# Find the comparables: real_acc.txt

The file `real_acc.txt` contains important property information like number total appraised value (the target on this exercise), neighborhood, school district, economic group, land value, and more. Let's load this file and grab a subset with the important columns to continue our study.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd

from src.definitions import ROOT_DIR
from src.data.utils import Table, save_pickle

In [3]:
real_acct_fn = ROOT_DIR / 'data/external/2016/Real_acct_owner/real_acct.txt'
assert real_acct_fn.exists()

In [4]:
real_acct = Table(real_acct_fn, '2016')

In [5]:
real_acct.get_header()

['acct',
 'yr',
 'mailto',
 'mail_addr_1',
 'mail_addr_2',
 'mail_city',
 'mail_state',
 'mail_zip',
 'mail_country',
 'undeliverable',
 'str_pfx',
 'str_num',
 'str_num_sfx',
 'str',
 'str_sfx',
 'str_sfx_dir',
 'str_unit',
 'site_addr_1',
 'site_addr_2',
 'site_addr_3',
 'state_class',
 'school_dist',
 'map_facet',
 'key_map',
 'Neighborhood_Code',
 'Neighborhood_Grp',
 'Market_Area_1',
 'Market_Area_1_Dscr',
 'Market_Area_2',
 'Market_Area_2_Dscr',
 'econ_area',
 'econ_bld_class',
 'center_code',
 'yr_impr',
 'yr_annexed',
 'splt_dt',
 'dsc_cd',
 'nxt_bld',
 'bld_ar',
 'land_ar',
 'acreage',
 'Cap_acct',
 'shared_cad',
 'land_val',
 'bld_val',
 'x_features_val',
 'ag_val',
 'assessed_val',
 'tot_appr_val',
 'tot_mkt_val',
 'prior_land_val',
 'prior_bld_val',
 'prior_x_features_val',
 'prior_ag_val',
 'prior_tot_appr_val',
 'prior_tot_mkt_val',
 'new_construction_val',
 'tot_rcn_val',
 'value_status',
 'noticed',
 'notice_dt',
 'protested',
 'certified_date',
 'rev_dt',
 'rev_by',
 '

# Load accounts and columns of interest
Let's remove the account numbers that don't meet free-standing single-family home criteria that we found while processing the `building_res.txt` file.

Also, the columns above show a lot of value information along property groups that might come in handy when predicting the appraised value. Now let's get a slice of some of the important columns.

In [6]:
skiprows = real_acct.get_skiprows()

In [7]:
cols = [
    'acct',
    'site_addr_3', # Zip
    'school_dist',
    'Neighborhood_Code',
    'Market_Area_1_Dscr',
    'Market_Area_2_Dscr',
    'center_code',
    'bld_ar',
    'land_ar',
    'acreage',
    'land_val',
    'tot_appr_val', # Target
    'prior_land_val',
    'prior_tot_appr_val',
    'new_own_dt',  # New owner date
    'lgl_1', # Block and lot in the neighborhood
    'lgl_2', # Official subdivision name
]

In [8]:
real_acct_df = real_acct.get_df(skiprows=skiprows, usecols=cols)

In [9]:
real_acct_df.head()

,acct,site_addr_3,school_dist,Neighborhood_Code,Market_Area_1_Dscr,Market_Area_2_Dscr,center_code,bld_ar,land_ar,acreage,land_val,tot_appr_val,prior_land_val,prior_tot_appr_val,new_own_dt,lgl_1,lgl_2
0,21440000001,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,2537,5000,0.1148,125000.0,145200.0,75000.0,132000.0,2012-09-11 00:00:00.000,LT 1 BLK 426,SSBB
1,21470000008,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,1000,5000,0.1148,74900.0,75000.0,64000.0,65000.0,1988-01-02 00:00:00.000,LT 8 BLK 429,SSBB
2,21480000002,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,1496,5000,0.1148,125000.0,85929.0,75000.0,78118.0,2004-07-28 00:00:00.000,LT 2 BLK 430,SSBB
3,21650000007,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,3387,5000,0.1148,125000.0,549004.0,75000.0,75000.0,2013-10-14 00:00:00.000,LT 7 BLK 447,SSBB
4,21650000011,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,1508,6250,0.1435,140625.0,181370.0,84375.0,164882.0,2001-05-05 00:00:00.000,LT 11 BLK 447,SSBB


Double check if the there is only one account number per row

In [10]:
assert real_acct_df['acct'].is_unique

# Describe and clean the columns

Now we must describe each column by answering:

* Meaning
* Descriptive statistics or value counts
* Data type

There is no explicit document provided by HCAD explaining all the variables, but most are easy to guess for using their name.

## Fix column names
We would like the column names to be all lower case, with no spaces nor non-alphanumeric characters.

In [11]:
from src.data.utils import fix_column_names

In [12]:
real_acct_df.columns

Index(['acct', 'site_addr_3', 'school_dist', 'Neighborhood_Code',
       'Market_Area_1_Dscr', 'Market_Area_2_Dscr', 'center_code', 'bld_ar',
       'land_ar', 'acreage', 'land_val', 'tot_appr_val', 'prior_land_val',
       'prior_tot_appr_val', 'new_own_dt', 'lgl_1', 'lgl_2'],
      dtype='object')

In [13]:
real_acct_df = fix_column_names(real_acct_df)

In [14]:
real_acct_df.columns

Index(['acct', 'site_addr_3', 'school_dist', 'neighborhood_code',
       'market_area_1_dscr', 'market_area_2_dscr', 'center_code', 'bld_ar',
       'land_ar', 'acreage', 'land_val', 'tot_appr_val', 'prior_land_val',
       'prior_tot_appr_val', 'new_own_dt', 'lgl_1', 'lgl_2'],
      dtype='object')

### Find duplicated rows

In [15]:
cond0 = real_acct_df.duplicated()
real_acct_df.loc[cond0, :]

,acct,site_addr_3,school_dist,neighborhood_code,market_area_1_dscr,market_area_2_dscr,center_code,bld_ar,land_ar,acreage,land_val,tot_appr_val,prior_land_val,prior_tot_appr_val,new_own_dt,lgl_1,lgl_2


## Zip code: site_addr_3
These are the property zip code. They should be non-zero integers. Most should start with 77.

In [16]:
# Let's change the column name
real_acct_df.rename({'site_addr_3': 'zip_code'}, axis=1, inplace=True)

In [17]:
# there were some empty zip code: ' '
real_acct_df['zip_code'] = real_acct_df['zip_code'].apply(lambda code: code if code.startswith('77') else np.nan )

In [18]:
real_acct_df['zip_code'] = pd.to_numeric(real_acct_df['zip_code'], downcast='unsigned')

In [19]:
real_acct_df['zip_code'].value_counts()

77449.0    30333
77084.0    24708
77429.0    24006
77433.0    23831
77379.0    23470
           ...  
77357.0       99
77002.0       87
77365.0       73
77535.0        7
77443.0        4
Name: zip_code, Length: 140, dtype: int64

In [20]:
real_acct_df['zip_code'].describe()

count    957645.000000
mean      77205.187268
std         194.750499
min       77002.000000
25%       77045.000000
50%       77085.000000
75%       77389.000000
max       77598.000000
Name: zip_code, dtype: float64

In [21]:
print(f"The number of missing values is: {sum(real_acct_df['zip_code'].isnull())}")

The number of missing values is: 46


## School district: school_dist
The school district values come coded as two digit integers. There are a few numbers missing from the code sequence, i.e. 10, 11, 13, 14, and 22. This is consistent with the ISD codes in the `code_jur_list` table. Let's decode the school district values using this table, but first let's represent the missing data with NaNs.

In [22]:
from src.data.utils import decode_isd

In [23]:
real_acct_df['school_dist'].head()

0    01   
1    01   
2    01   
3    01   
4    01   
Name: school_dist, dtype: object

In [24]:
real_acct_df['school_dist'] = decode_isd(real_acct_df['school_dist'])

In [25]:
real_acct_df['school_dist'].value_counts(normalize=True)

HOUSTON ISD                     0.281761
CYPRESS-FAIRBANKS ISD           0.154690
KLEIN ISD                       0.073475
HUMBLE ISD                      0.057477
KATY ISD                        0.053951
ALDINE ISD                      0.053262
PASADENA ISD                    0.051970
SPRING ISD                      0.042361
CLEAR CREEK ISD                 0.037231
ALIEF ISD                       0.035638
SPRING BRANCH ISD               0.033610
GOOSE CREEK CISD                0.024351
TOMBALL ISD                     0.020237
GALENA PARK ISD                 0.018560
DEER PARK ISD                   0.014386
LA PORTE ISD                    0.013354
CHANNELVIEW ISD                 0.008916
SHELDON ISD                     0.008255
CROSBY ISD                      0.007619
HUFFMAN ISD                     0.004609
WALLER ISD                      0.003125
PEARLAND ISD                    0.000662
NEW CANEY ISD                   0.000403
STAFFORD MSD                    0.000089
DAYTON ISD      

In [26]:
print(f"The number of missing values is: {sum(real_acct_df['school_dist'].isnull())}")

The number of missing values is: 55


## neighborhood_code

From [Definition help](https://pdata.hcad.org/Desc/Definition_help.pdf) file:
> Residential valuation neighborhoods are groups of comparable properties whose
boundaries were developed based on location and similarity of property data
characteristics. Each neighborhood in a school district has a unique identifier known as a
residential valuation number. These neighborhood boundaries are maintained via an ongoing office and field review. As neighborhoods change, neighborhood lines are redrawn
to reflect the changes, and maintain the homogeneity of the neighborhood. 

In [27]:
from src.data.utils import decode_nhood

In [28]:
real_acct_df['neighborhood_code'] = decode_nhood(real_acct_df['neighborhood_code'])

No description for code: 9831.06


In [29]:
real_acct_df['neighborhood_code'].value_counts(normalize=True)

M/R                              0.004535
OAK MEADOWS                      0.002035
RED BLUFF  TERRACE SEC 1-7       0.001802
HOUSTON HEIGHTS                  0.001684
BEAR CREEK VLG 2,3,4,6/OTHERS    0.001617
                                   ...   
TWIN OAKS MOBILE EST. U/R R/P    0.000001
HAYDEN LAKES SEC 8               0.000001
TWELVE OAKS SEC 1                0.000001
GRAMERCY PARK TH                 0.000001
TALL CEDARS II (MH)              0.000001
Name: neighborhood_code, Length: 6015, dtype: float64

In [30]:
print(f"The number of missing values is: {sum(real_acct_df['neighborhood_code'].isnull())}")

The number of missing values is: 1


## Market area description: market_area_1_dscr

In [31]:
real_acct_df['market_area_2_dscr'].value_counts(normalize=True)

ISD 17 - Klein ISD                                    0.073422
ISD 21 - Pasadena ISD                                 0.052091
ISD 04 - Far Northwest, Outside Hwy 6                 0.049492
1C South of Old Spanish Trail btwn SH 288 and I-45    0.041510
ISD 04 - Far West, South of US 290/North of FM 529    0.035591
                                                        ...   
ISD 29 - Pearland ISD                                 0.000662
ISD 07 - New Caney ISD                                0.000403
ISD 31 - Stafford MSD                                 0.000088
ISD 28 - Dayton ISD                                   0.000006
FM 1960 West                                          0.000001
Name: market_area_2_dscr, Length: 72, dtype: float64

In [32]:
cond0 = real_acct_df['market_area_2_dscr'].str.strip() == ''
real_acct_df.loc[cond0, :]

,acct,zip_code,school_dist,neighborhood_code,market_area_1_dscr,market_area_2_dscr,center_code,bld_ar,land_ar,acreage,land_val,tot_appr_val,prior_land_val,prior_tot_appr_val,new_own_dt,lgl_1,lgl_2


In [33]:
print(f"The number of missing values is: {sum(real_acct_df['market_area_2_dscr'].isnull())}")

The number of missing values is: 0


## center_code
From [Definition help](https://pdata.hcad.org/Desc/Definition_help.pdf) file:
> The center code indicates the individual (HCAD employee or contractor) that is
responsible for working and valuing the property account.

In [34]:
real_acct_df['center_code'].value_counts()

10       148162
61       118331
50        99013
72        92252
20        92175
90        85802
40        78670
60        65816
30        58467
80        35301
70        32177
71        29435
81        20504
32         1092
            378
33           50
CC           16
27           15
45            9
21            9
22            3
25            3
51            3
RB            2
43            2
28            1
35            1
NV            1
62            1
Name: center_code, dtype: int64

In [35]:
real_acct_df['center_code'] = real_acct_df['center_code'].apply(lambda x: np.nan if x.isspace() else x)

In [36]:
print(f"The number of missing values is: {sum(real_acct_df['center_code'].isnull())}")

The number of missing values is: 378


## Building area: 'bld_ar'
This seems to be the total area occupied by buildings in the parcel. It possibly includes non-livable buildings like barns, sheds, and other outbuildings.

In [37]:
from src.data.utils import fix_area_column

In [38]:
real_acct_df = fix_area_column(real_acct_df, 'bld_ar')

Values less than 100 sqft: 7
1917      0
31801     0
47618     0
131847    0
131881    0
148243    0
165247    0
Name: bld_ar, dtype: uint16
Since the column contains NaNs, it can't be casted as int type


The new data type is: float64


The number of null values is: 7


bld_ar description:
count    957684.000000
mean       2112.095100
std         954.911264
min         120.000000
25%        1464.000000
50%        1906.000000
75%        2537.000000
max       26401.000000
Name: bld_ar, dtype: float64


## Land area: land_ar
Land area in square feet.

In [39]:
real_acct_df = fix_area_column(real_acct_df, 'land_ar')

Values less than 100 sqft: 1032
771       0
1584      0
2088      0
4481      0
5135      0
         ..
956217    0
956235    0
956371    0
956409    0
957244    0
Name: land_ar, Length: 1032, dtype: uint64
Since the column contains NaNs, it can't be casted as int type


The new data type is: float64


The number of null values is: 1032


land_ar description:
count    9.566590e+05
mean     1.653732e+04
std      4.844536e+06
min      3.510000e+02
25%      6.086000e+03
50%      7.360000e+03
75%      9.340000e+03
max      4.544441e+09
Name: land_ar, dtype: float64


## Land appraised value: land_val
Appraised value of the land. It is represented in US dollars, should integers unless there are NaNs. 

In [40]:
real_acct_df['land_val'].head()

0    125000.0
1     74900.0
2    125000.0
3    125000.0
4    140625.0
Name: land_val, dtype: float64

In [41]:
print(f"The number of missing values is: {sum(real_acct_df['land_val'].isnull())}")

The number of missing values is: 4


In [42]:
real_acct_df['land_val'].describe().apply(lambda x: format(x, 'f'))

count      957687.000000
mean        74993.389017
std        175567.440264
min             0.000000
25%         19961.000000
50%         27431.000000
75%         48307.500000
max      14598650.000000
Name: land_val, dtype: object

## Total appraised value: tot_appr_val
This is the target to be predicted. It is represented in US dollars, should integers unless there are NaNs. 

In [43]:
from src.data.utils import fix_appraised_values

In [44]:
real_acct_df = fix_appraised_values(real_acct_df, 'tot_appr_val')

tot_appr_val: head
0    145200.0
1     75000.0
2     85929.0
3    549004.0
4    181370.0
Name: tot_appr_val, dtype: float64


The number of missing values is: 4


tot_appr_val: describe
count      957687.000000
mean       219212.465773
std        279995.209203
min           100.000000
25%        100000.000000
50%        150262.000000
75%        234458.000000
max      17266818.000000
Name: tot_appr_val, dtype: object


## Prior land value: prior_land_val
Last year's appraised land value. It is represented in US dollars, should integers unless there are NaNs.

In [45]:
real_acct_df = fix_appraised_values(real_acct_df, 'prior_land_val')

prior_land_val: head
0    75000.0
1    64000.0
2    75000.0
3    75000.0
4    84375.0
Name: prior_land_val, dtype: float64


The number of missing values is: 1666


prior_land_val: describe
count      956025.000000
mean        71895.343272
std        170130.801341
min             0.000000
25%         18585.000000
50%         25600.000000
75%         46018.000000
max      14598650.000000
Name: prior_land_val, dtype: object


## Prior total appraised value: prior_tot_appr_val
Last year's total appraised land value. It is represented in US dollars, should integers unless there are NaNs.

In [46]:
real_acct_df = fix_appraised_values(real_acct_df, 'prior_tot_appr_val')

prior_tot_appr_val: head
0    132000.0
1     65000.0
2     78118.0
3     75000.0
4    164882.0
Name: prior_tot_appr_val, dtype: float64


The number of missing values is: 1666


prior_tot_appr_val: describe
count      956025.000000
mean       202540.310016
std        263124.394608
min             0.000000
25%         90491.000000
50%        137500.000000
75%        215582.000000
max      15697108.000000
Name: prior_tot_appr_val, dtype: object


## New owner date: new_own_dt
When was the last recorded ownership changed on the property. Should be datetime type.

In [47]:
real_acct_df['new_own_dt'] = pd.to_datetime(real_acct_df['new_own_dt'])

In [48]:
print(f"The number of missing values is: {sum(real_acct_df['new_own_dt'].isnull())}")

The number of missing values is: 9


In [49]:
real_acct_df['new_own_dt']

0        2012-09-11
1        1988-01-02
2        2004-07-28
3        2013-10-14
4        2001-05-05
            ...    
957686   2005-01-02
957687   2005-05-31
957688   1988-01-02
957689   2013-11-06
957690   2016-09-13
Name: new_own_dt, Length: 957691, dtype: datetime64[ns]

In [50]:
real_acct_df['new_own_dt'].describe(datetime_is_numeric=True)

count                           957682
mean     2005-11-13 14:47:46.416200704
min                1955-05-13 00:00:00
25%                2000-05-26 00:00:00
50%                2007-07-16 00:00:00
75%                2013-06-28 00:00:00
max                2017-10-06 00:00:00
Name: new_own_dt, dtype: object

# Export real_acct

In [51]:
save_fn = ROOT_DIR / 'data/raw/2016/real_acct_comps.pickle'
save_pickle(real_acct_df, save_fn)